In [1]:
import requests
import time
import concurrent.futures
import threading
import asyncio
import aiohttp
import nest_asyncio

#### Wersja synchroniczna

In [2]:
def download_site(url, session):
    session.get(url)


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    async_duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {async_duration} seconds")

Downloaded 160 in 15.463999032974243 seconds


#### Wersja wątkowa

In [3]:
thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    session.get(url) 


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)

start_time = time.time()
download_all_sites(sites)
thread_duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {thread_duration} seconds")

Downloaded 160 in 3.357004165649414 seconds


#### Wersja asyncio

In [4]:
async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(session.get(url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)

        
start_time = time.time()
nest_asyncio.apply()
asyncio.run(download_all_sites(sites))
asyncio_duration = time.time() - start_time
print(f"Downloaded {len(sites)} sites in {asyncio_duration} seconds")

Downloaded 160 sites in 0.9400002956390381 seconds


#### Wersja synchroniczna zależna od CPU

In [5]:
def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    for number in numbers:
        cpu_bound(number)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    cpusynth_duration = time.time() - start_time
    print(f"Duration {cpusynth_duration} seconds")

Duration 13.136292934417725 seconds


#### Wersja wątkowa zależna od CPU 

In [6]:
def find_sums(numbers):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(cpu_bound, numbers)


start_time = time.time()
find_sums(numbers)
cputh_duration = time.time() - start_time
print(f"Duration {cputh_duration} seconds")

Duration 14.25199556350708 seconds


#### Wersja asynchroniczna zależna od CPU

In [7]:
async def cpu_bound_as(number):
    return sum(i * i for i in range(number))

async def find_sums_as(numbers):
    tasks = []
    for number in numbers:
        task = asyncio.ensure_future(cpu_bound_as(number))
        tasks.append(task)
    await asyncio.gather(*tasks, return_exceptions=True)

        
start_time = time.time()
nest_asyncio.apply()
asyncio.run(find_sums_as(numbers))
cpuasynth_duration = time.time() - start_time
print(f"Duration {cpuasynth_duration} seconds")

Duration 12.337998628616333 seconds


#### Porównanie

In [9]:
print(f"Wersja asynchroniczna: {async_duration} sekund")
print(f"Wersja wątkowa: {thread_duration} sekund")
print(f"Wrsja asynchroniczna: {asyncio_duration} sekund\n")
print(f"Wersja synchroniczna zal. od CPU: {cpusynth_duration} sekund")
print(f"Wersja wątkowa zal. od CPU: {cputh_duration} sekund")
print(f"Wersja asynchroniczna zal. od CPU: {cpuasynth_duration} sekund")

Wersja asynchroniczna: 15.463999032974243 sekund
Wersja wątkowa: 3.357004165649414 sekund
Wrsja asynchroniczna: 0.9400002956390381 sekund

Wersja synchroniczna zal. od CPU: 13.136292934417725 sekund
Wersja wątkowa zal. od CPU: 14.25199556350708 sekund
Wersja asynchroniczna zal. od CPU: 12.337998628616333 sekund


Niestety multiprocessing nie działa na jupyterze